In [181]:
import pyarrow as pa
import pyarrow.flight as fl

# client = pa.flight.connect("grpc://0.0.0.0:8815")


In [ ]:
# Upload a new dataset
data_table = pa.table(
    [["Mario", "Luigi", "Peach"]],
    names=["Character"]
)
upload_descriptor = pa.flight.FlightDescriptor.for_path("characters/")
writer, _ = client.do_put(upload_descriptor, data_table.schema)
writer.write_table(data_table)
writer.close()

In [ ]:
upload_descriptor

In [ ]:
info = client.get_flight_info(upload_descriptor)

In [ ]:
info.endpoints[0].ticket

In [ ]:
reader = client.do_get(info.endpoints[0].ticket)
table = reader.read_all()
table.to_pandas()

## A more serious test with more data

In [194]:
import pyarrow.parquet as pq
housing = pq.read_table("../housing_market/clean_export_pararius_24122021.parquet")
housing.to_pandas().head()

,link,price,house_size,interior,number_of_rooms,construction_period,postal_code,neighbourhood,housing_type,street,for_rent
28,https://www.pararius.com/apartment-for-rent/am...,2250.0,140,Furnished,5,<NA>,1019 AX,Oostelijk Havengebied,Apartment,Panamakade,True
26,https://www.pararius.com/apartment-for-rent/am...,2150.0,110,Furnished,3,<NA>,1058 DS,Westindische Buurt,Apartment,Postjeskade,True
7,https://www.pararius.com/apartment-for-rent/am...,1695.0,88,Upholstered,4,<NA>,1078 MS,Scheldebuurt,Apartment,Biesboschstraat,True
26,https://www.pararius.com/apartment-for-rent/am...,1500.0,45,Upholstered,2,<NA>,1051 BX,Staatsliedenbuurt,Apartment,Van Hogendorpstraat 166 II,True
29,https://www.pararius.com/apartment-for-rent/am...,1500.0,80,None,2,1930,1052 KC,Frederik Hendrikbuurt,Apartment,Van Oldenbarneveldtstraat,True


In [ ]:
upload_descriptor = pa.flight.FlightDescriptor.for_path("housing")
writer, _ = client.do_put(upload_descriptor, housing.schema)
writer.write_table(housing)
writer.close()

In [ ]:
endpoints = client.get_flight_info(upload_descriptor).endpoints
reader = client.do_get(endpoints[0].ticket)
table = reader.read_all()
table.to_pandas().head()

In [ ]:
endpoints[0].ticket

In [ ]:
for flight in client.list_flights():
    descriptor = flight.descriptor
    print("Path:", descriptor.path[0].decode('utf-8'), "Rows:", flight.total_records, "Size:", flight.total_bytes)
    print("=== Schema ===")
    print(flight.schema)
    print("==============")
    print("")

## DeltaServer test

In [186]:
client = pa.flight.connect("grpc://0.0.0.0:8815")

In [202]:
f = client.get_flight_info(fl.FlightDescriptor.for_path("housing"))
f.total_records

1208

In [208]:
f.endpoints[0].ticket.ticket

b'./delta_storage/housing'

In [213]:
t = fl.Ticket(ticket="./delta_storage/housing")
print(t.ticket)
client.do_get(ticket=t).read_all().to_pandas()

TypeError: __init__() got an unexpected keyword argument 'table_path'

In [185]:
# List all Delta Tables
for flight in client.list_flights():
    descriptor = flight.descriptor
    print("Path:", descriptor.path[0].decode('utf-8'), "Rows:", flight.total_records, "Size:", flight.total_bytes)
    print("=== Schema ===")
    print(flight.schema)
    print("==============")
    print("")


Path: ./delta_storage/housing Rows: 1208 Size: 6645
=== Schema ===
link: string
price: float
house_size: int32
interior: string
number_of_rooms: int32
construction_period: int64
postal_code: string
neighbourhood: string
housing_type: string
street: string
for_rent: bool
__index_level_0__: int64



In [193]:
flight_descriptor = pa.flight.FlightDescriptor.for_path("housing/")
endpoint = client.get_flight_info(flight_descriptor).endpoints[0]

FlightUnavailableError: Flight returned unavailable error, with message: failed to connect to all addresses; last error: UNKNOWN: ipv4:0.0.0.0:8815: Failed to connect to remote host: Connection refused

In [178]:
table = client.do_get(endpoint.ticket).read_all()
table.to_pandas()

,link,price,house_size,interior,number_of_rooms,construction_period,postal_code,neighbourhood,housing_type,street,for_rent,__index_level_0__
0,https://www.pararius.com/apartment-for-rent/am...,2250.0,140,Furnished,5,NaN,1019 AX,Oostelijk Havengebied,Apartment,Panamakade,True,28
1,https://www.pararius.com/apartment-for-rent/am...,2150.0,110,Furnished,3,NaN,1058 DS,Westindische Buurt,Apartment,Postjeskade,True,26
2,https://www.pararius.com/apartment-for-rent/am...,1695.0,88,Upholstered,4,NaN,1078 MS,Scheldebuurt,Apartment,Biesboschstraat,True,7
3,https://www.pararius.com/apartment-for-rent/am...,1500.0,45,Upholstered,2,NaN,1051 BX,Staatsliedenbuurt,Apartment,Van Hogendorpstraat 166 II,True,26
4,https://www.pararius.com/apartment-for-rent/am...,1500.0,80,None,2,1930.0,1052 KC,Frederik Hendrikbuurt,Apartment,Van Oldenbarneveldtstraat,True,29
...,...,...,...,...,...,...,...,...,...,...,...,...
1203,https://www.pararius.com/studio-for-rent/amste...,655.0,30,Upholstered,1,NaN,1102 BL,"Bijlmer Centrum (D,F,H)",Studio,Elizabeth Cady Stantonplein,True,25
1204,https://www.pararius.com/studio-for-rent/amste...,1750.0,50,Upholstered or furnished,2,NaN,1054 XR,Overtoomse Sluis,Studio,Kanaalstraat 168 4,True,15
1205,https://www.pararius.com/studio-for-rent/amste...,1500.0,39,Furnished,1,NaN,1034 DA,Banne Buiksloot,Studio,Bezaanjachtplein,True,19
1206,https://www.pararius.com/studio-for-rent/amste...,1000.0,35,Upholstered,1,NaN,1043 CP,Bedrijventerrein Sloterdijk,Studio,Barajasweg,True,26


In [183]:
with pa.flight.connect("grpc://0.0.0.0:8815") as cl:
    ticket = fl.Ticket("delta_storage/housing/")
    df = client.do_get(ticket).read_all().to_pandas()

df

,link,price,house_size,interior,number_of_rooms,construction_period,postal_code,neighbourhood,housing_type,street,for_rent,__index_level_0__
0,https://www.pararius.com/apartment-for-rent/am...,2250.0,140,Furnished,5,NaN,1019 AX,Oostelijk Havengebied,Apartment,Panamakade,True,28
1,https://www.pararius.com/apartment-for-rent/am...,2150.0,110,Furnished,3,NaN,1058 DS,Westindische Buurt,Apartment,Postjeskade,True,26
2,https://www.pararius.com/apartment-for-rent/am...,1695.0,88,Upholstered,4,NaN,1078 MS,Scheldebuurt,Apartment,Biesboschstraat,True,7
3,https://www.pararius.com/apartment-for-rent/am...,1500.0,45,Upholstered,2,NaN,1051 BX,Staatsliedenbuurt,Apartment,Van Hogendorpstraat 166 II,True,26
4,https://www.pararius.com/apartment-for-rent/am...,1500.0,80,None,2,1930.0,1052 KC,Frederik Hendrikbuurt,Apartment,Van Oldenbarneveldtstraat,True,29
...,...,...,...,...,...,...,...,...,...,...,...,...
1203,https://www.pararius.com/studio-for-rent/amste...,655.0,30,Upholstered,1,NaN,1102 BL,"Bijlmer Centrum (D,F,H)",Studio,Elizabeth Cady Stantonplein,True,25
1204,https://www.pararius.com/studio-for-rent/amste...,1750.0,50,Upholstered or furnished,2,NaN,1054 XR,Overtoomse Sluis,Studio,Kanaalstraat 168 4,True,15
1205,https://www.pararius.com/studio-for-rent/amste...,1500.0,39,Furnished,1,NaN,1034 DA,Banne Buiksloot,Studio,Bezaanjachtplein,True,19
1206,https://www.pararius.com/studio-for-rent/amste...,1000.0,35,Upholstered,1,NaN,1043 CP,Bedrijventerrein Sloterdijk,Studio,Barajasweg,True,26
